In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import scipy as sp
from scipy import stats

In [ ]:
w=pd.read_csv("/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/7/w.tsv", sep='\t', index_col=0)
w2=pd.DataFrame(sp.stats.zscore(w, axis=1))
w2.index=w.index
w2.columns=w.columns

In [ ]:
w2_2 = w2[['F0', 'F1', 'F2', 'F3', 'F4', 'F5']]

In [ ]:
df_indexmax = pd.DataFrame(w2_2.idxmax(axis=1))

In [ ]:
df_indexmax.columns = ["max_state"]

In [ ]:
df_indexmax.value_counts("max_state")

In [ ]:
salient_F0_gene_list = df_indexmax.loc[df_indexmax.max_state == "F0",].index.tolist()
salient_F1_gene_list = df_indexmax.loc[df_indexmax.max_state == "F1",].index.tolist()
salient_F2_gene_list = df_indexmax.loc[df_indexmax.max_state == "F2",].index.tolist()
salient_F3_gene_list = df_indexmax.loc[df_indexmax.max_state == "F3",].index.tolist()
salient_F4_gene_list = df_indexmax.loc[df_indexmax.max_state == "F4",].index.tolist()
salient_F5_gene_list = df_indexmax.loc[df_indexmax.max_state == "F5",].index.tolist()

In [ ]:
# Define the folder path
folder_path = "/Volumes/SSD_Yue/MsigDB_genesets/hallmark_genesets_2024/"

# Initialize a dictionary to store the DataFrames
MsigDB_hallmark_pathways = {}

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a TSV file
    if filename.endswith(".tsv"):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep='\t')
        genes_cell = df.iloc[16, 1]  # Adjust the row and column as necessary
        gene_list = genes_cell.split(',')
        key = os.path.splitext(filename)[0]
        MsigDB_hallmark_pathways[key] = gene_list

# Accessing the DataFrames
for key, gene_list in MsigDB_hallmark_pathways.items():
    print(f"pathway for {key}:")

In [ ]:
lists_to_compare = [salient_F0_gene_list, salient_F1_gene_list, salient_F2_gene_list, salient_F3_gene_list, salient_F4_gene_list, salient_F5_gene_list]

overlap_results = []

# For each of the 6 lists
for i, comp_list in enumerate(lists_to_compare, start=0):
    # Compare with each list in the dictionary
    for key, dict_list in MsigDB_hallmark_pathways.items():
        overlap = set(comp_list) & set(dict_list)  # Find intersection
        overlap_results.append({
            "Comparison List": f"list_F{i}",
            "Dictionary List": key,
            "Overlap": list(overlap),
            "Overlap Count": len(overlap)  # Optional: size of the overlap
        })

# Convert results to a DataFrame
overlap_df = pd.DataFrame(overlap_results)

In [ ]:
pd.set_option('display.max_rows', None)
overlap_df

In [ ]:
#We get rid of the biologically "vague" pathway gene sets
overlap_df_filtered = overlap_df[~overlap_df['Dictionary List'].isin([
    'HALLMARK_UNFOLDED_PROTEIN_RESPONSE.v2024.1.Hs','HALLMARK_SPERMATOGENESIS.v2024.1.Hs',
    'HALLMARK_ALLOGRAFT_REJECTION.v2024.1.Hs','HALLMARK_APICAL_JUNCTION.v2024.1.Hs',
    'HALLMARK_APICAL_SURFACE.v2024.1.Hs','HALLMARK_ADIPOGENESIS.v2024.1.Hs','HALLMARK_COAGULATION.v2024.1.Hs','HALLMARK_COMPLEMENT.v2024.1.Hs',
    'HALLMARK_CHOLESTEROL_HOMEOSTASIS.v2024.1.Hs','HALLMARK_BILE_ACID_METABOLISM.v2024.1.Hs',
    'HALLMARK_HEME_METABOLISM.v2024.1.Hs','HALLMARK_INFLAMMATORY_RESPONSE.v2024.1.Hs','HALLMARK_MITOTIC_SPINDLE.v2024.1.Hs',
    'HALLMARK_MYOGENESIS.v2024.1.Hs','HALLMARK_OXIDATIVE_PHOSPHORYLATION.v2024.1.Hs',
    'HALLMARK_PANCREAS_BETA_CELLS.v2024.1.Hs','HALLMARK_PEROXISOME.v2024.1.Hs',
    'HALLMARK_PROTEIN_SECRETION.v2024.1.Hs','HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY.v2024.1.Hs',
    'HALLMARK_XENOBIOTIC_METABOLISM.v2024.1.Hs'
])]

In [ ]:
overlap_df_filtered.loc[overlap_df_filtered['Comparison List'] == 'list_F2',:].sort_values("Overlap Count", ascending=False)